<a href="https://colab.research.google.com/github/dchirosca/bot-detection-econophysics/blob/main/bot_detection_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(readr)
library(dplyr)
library(ggplot2)
library(corrplot)
library(caret)
library(class)
library(xgboost)
library(Matrix)
library(iml)
options(scipen = 99)

# Data pre-processing -----------------------------------------------------

# Keeping the columns relevant for analysis
col_to_keep<-c("created_at", "description", "favourites_count", "followers_count", "friends_count", "geo_enabled", "id", "screen_name", "statuses_count",
               "average_tweets_per_day", "account_type")
dataset<-subset(dataset, select = col_to_keep)
dataset$account_type<-as.factor(dataset$account_type)

dataset$geo_enabled<-ifelse(dataset$geo_enabled == "TRUE", 1, 0)

description_char_l<-ifelse(is.na(dataset$description) == TRUE, 0, nchar(dataset$description))
description_no_words<-c()
for(i in 1:dim(dataset)[1]){
  if(is.na(dataset$description[i]) == TRUE){
    description_no_words[i]<-0
  } else{
    description_no_words[i]<-length(strsplit(dataset$description[i], " ")[[1]])
  }
}
dataset<-cbind(dataset, description_char_l, description_no_words)

dt<-dataset
# Removing rows with 0 for numeric variables
dt<-dt %>% filter(followers_count>0)
dt<-dt %>% filter(favourites_count>0)
dt<-dt %>% filter(friends_count>0)
dt<-dt %>% filter(statuses_count>0)
dt<-dt %>% filter(average_tweets_per_day>0)
dt<-dt %>% filter(description_char_l>0)
dt<-dt %>% filter(description_no_words>0)
dt$average_tweets_per_day<-ceiling(dt$average_tweets_per_day)

dt_bot<-dt %>% filter(account_type == "bot")
dt_human<-dt %>% filter(account_type == "human")

# Calculating Benford's law -----------------------------------------------

benfords_distribution<-log10(1 + 1/(1:9))
extract_leading_digit<-function(x) {
  as.numeric(substr(as.character(x), 1, 1))
}

benfords_law<-function(variable){
  leading_digits<-sapply(variable, extract_leading_digit)
  digit_distribution<-table(leading_digits)/length(leading_digits)
  observed<-as.numeric(digit_distribution*length(leading_digits))
  names(observed)<-names(digit_distribution)
  if(length(names(observed)) < 9){
    vector_of_z<-rep(0, times = 9-length(names(observed)))
    observed<-c(observed, vector_of_z)
    names(observed)<-c(1:9)
  }
  chisq_test<-chisq.test(observed, p = benfords_distribution)
  if(chisq_test$p.value < 0.05){
    conclusion<-"The distribution significantly differs from Benford's law distribution"
  } else{
    conclusion<-"The distribution DOES NOT significantly differ from Benford's law distribution"
  }
  return(conclusion)
}


# Benford's law study for the entire dataset -----------------------------------------------------

benfords_law(dt$followers_count)
benfords_law(dt$favourites_count)
benfords_law(dt$friends_count)
benfords_law(dt$statuses_count)
benfords_law(dt$average_tweets_per_day)
benfords_law(dt$description_char_l)
benfords_law(dt$description_no_words)

# Benford's law study for the human dataset -----------------------------------------------------

benfords_law(dt_human$followers_count)

benfords_law(dt_human$favourites_count) # The distribution DOES NOT significantly differ from Benford's law distribution
leading_digits<-sapply(dt_human$favourites_count, extract_leading_digit)
digit_distribution<-table(leading_digits)/length(leading_digits)
barplot(digit_distribution, main = "Distributia primei cifre pentru variabila de tweeturi favorite",
        names.arg = 1:9,
        ylim = c(0, max(digit_distribution, benfords_distribution)),
        col = "#619cff")
points(1:9, benfords_distribution, col = "black", pch = 21, cex = 1.8)

benfords_law(dt_human$friends_count)

benfords_law(dt_human$statuses_count) # The distribution DOES NOT significantly differ from Benford's law distribution
leading_digits<-sapply(dt_human$statuses_count, extract_leading_digit)
digit_distribution<-table(leading_digits)/length(leading_digits)
barplot(digit_distribution, main = "Distributia primei cifre pentru variabila de total statusuri",
        names.arg = 1:9,
        ylim = c(0, max(digit_distribution, benfords_distribution)),
        col = "#619cff")
points(1:9, benfords_distribution, col = "black", pch = 21, cex = 1.8)

benfords_law(dt_human$average_tweets_per_day)
benfords_law(dt_human$description_char_l)
benfords_law(dt_human$description_no_words)

# Benford's law study for the bot dataset -----------------------------------------------------

benfords_law(dt_bot$followers_count)
benfords_law(dt_bot$favourites_count)
benfords_law(dt_bot$friends_count)
benfords_law(dt_bot$statuses_count)
benfords_law(dt_bot$average_tweets_per_day)
benfords_law(dt_bot$description_char_l)
benfords_law(dt_bot$description_no_words)


# Exploratory data analysis -----------------------------------------------

account_count_plot<-ggplot(dataset)+
  geom_bar(aes(x=account_type, fill=account_type))+
  theme(legend.title=element_text(size=12,face="bold"), plot.title = element_text(size=16,face="bold",hjust = 0.5))+
  scale_color_manual(values=c("#f8766d", "#00ba38", "#619cff"))+
  labs(title="Tipul de conturi de Twitter din setul de date",x="Cluster", y = "Venit anual")
account_count_plot

favourites_plot<-ggplot(dataset, aes(x=account_type,y=favourites_count,fill=account_type))+
  geom_boxplot(outlier.colour="black", outlier.shape=16,outlier.size=2, notch=T)+
  theme(legend.title=element_text(size=12,face="bold"), plot.title = element_text(size=16,face="bold",hjust = 0.5))+
  scale_color_manual(values=c("#f8766d", "#00ba38", "#619cff"))+
  labs(title="Numarul de favourites in functie de tipul de cont",x="Tip cont", y = "Numar de favourites")
favourites_plot

statuses_plot<-ggplot(dataset, aes(x=account_type,y=statuses_count,fill=account_type))+
  geom_boxplot(outlier.colour="black", outlier.shape=16,outlier.size=2, notch=T)+
  theme(legend.title=element_text(size=12,face="bold"), plot.title = element_text(size=16,face="bold",hjust = 0.5))+
  scale_color_manual(values=c("#f8766d", "#00ba38", "#619cff"))+
  labs(title="Numarul de statusuri in functie de tipul de cont",x="Tip cont", y = "Numar de statusuri")
statuses_plot

# Correlation matrix for numeric variables
cor_matrix<-cor(dataset%>%select_if(is.numeric))
# Correlation plot
corrplot(cor_matrix, method = "circle")

cor_matrix_human<-cor(dt_human%>%select_if(is.numeric))
corrplot(cor_matrix_human, method = "circle")

cor_matrix_bot<-cor(dt_bot%>%select_if(is.numeric))
corrplot(cor_matrix_bot, method = "circle")

# Data mining -------------------------------------------------------------

# K-Nearest Neighbors -----------------------------------------------------

# Selecting relevant features and the target variable
features<-dataset[, -which(names(dataset) %in% c("account_type", "created_at", "description", "id", "screen_name", "description_char_l"))]
target<-dataset$account_type

# Splitting the dataset
set.seed(121)  # For reproducibility
splitIndex<-createDataPartition(target, p = 0.75, list = FALSE) # splitting the dataset in 80-20 training and test
train_data<-dataset[splitIndex,]
test_data<-dataset[-splitIndex,]

# Selecting relevant features and the target variable
features<-dataset[, -which(names(dataset) %in% c("account_type", "created_at", "description", "id", "screen_name", "description_char_l"))]
target<-dataset$account_type

# Splitting the dataset
set.seed(121)  # For reproducibility
splitIndex<-createDataPartition(target, p = 0.75, list = FALSE) # splitting the dataset in 80-20 training and test
train_data<-dataset[splitIndex,]
test_data<-dataset[-splitIndex,]

# Normalizing the data
preProcValues<-preProcess(train_data, method = c("center", "scale"))
train_data<-predict(preProcValues, train_data)
test_data<-predict(preProcValues, test_data)

# Extracting features and target variable from training and test sets
train_features <- train_data[, -which(names(train_data) %in% c("account_type", "created_at", "description", "id", "screen_name", "description_char_l"))]
train_target <- train_data$account_type
test_features <- test_data[, -which(names(test_data) %in% c("account_type", "created_at", "description", "id", "screen_name", "description_char_l"))]
test_target <- test_data$account_type

# Trying the KNN algorithm for different k values
accuracy<-c()
for (k in 1:20) {
  knn_model <- knn(train = train_features, test = test_features, cl = train_target, k = k)
  confusionMatrix<-table(test_target, knn_model)
  accuracy[k]<-sum(diag(confusionMatrix)) / sum(confusionMatrix)
}

accuracy_res<-data.frame(k = c(1:20), accuracy)
accuracy_res[which.max(accuracy_res$accuracy),] # the maximum accuracy is recorded for k = 10 neighbors

knn_model_k10<-knn(train = train_features, test = test_features, cl = train_target, k = 10)
confusionMatrix_k10<-table(test_target, knn_model)
confusionMatrix_k10
accuracy_k10<-sum(diag(confusionMatrix)) / sum(confusionMatrix)
accuracy_k10 # 0.8186772

barplot(accuracy, main = "Acuratetea algoritmilor KNN pentru diferite valori ale lui k",
        names.arg = 1:20,
        col = "#619cff")

# XGBoost -----------------------------------------------------------------

features<-dataset[, -which(names(dataset) %in% c("account_type", "created_at", "description", "id", "screen_name", "description_char_l"))]
target<-dataset$account_type

# Splitting the dataset
df<-dataset[, -which(names(dataset) %in% c("created_at", "description", "id", "screen_name", "description_char_l"))]
df$account_type<-ifelse(df$account_type == "bot", 1, 0)
set.seed(121)  # For reproducibility
splitIndex<-createDataPartition(target, p = 0.75, list = FALSE) # splitting the dataset in 80-20 training and test
train_data<-df[splitIndex,]
test_data<-df[-splitIndex,]

# Separate features and labels
train_label<-train_data$account_type
train_data<-train_data[, -which(names(train_data) == "account_type")]

test_label<-test_data$account_type
test_data<-test_data[, -which(names(test_data) == "account_type")]

dtrain<-xgb.DMatrix(data = as.matrix(train_data), label = train_label)
dtest<-xgb.DMatrix(data = as.matrix(test_data), label = test_label)

params<-list(
  objective = "binary:logistic",
  eta = 0.1,
  max_depth = 9,
  nthread = 2
)

set.seed(121)  # for reproducibility
xgb_model<-xgb.train(params, dtrain, nrounds = 10000)

# Predictions
preds<-predict(xgb_model, dtest)
preds<-ifelse(preds > 0.5, 1, 0)

# Confusion Matrix to evaluate performance
confusionMatrix(factor(preds), factor(test_label)) # 0.8704

importance_matrix <- xgb.importance(feature_names = colnames(train_data), model = xgb_model)
xgb.plot.importance(importance_matrix)